<a href="https://colab.research.google.com/github/akibkhan1/skin-lesion-classification/blob/main/ISIC2018_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modifying the Ground Truth CSV for Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
one_hot_labels = pd.read_csv('/content/ISIC2018_Task3_Training_GroundTruth.csv')
one_hot_labels.head(10)

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ISIC_0024311,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,ISIC_0024312,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,ISIC_0024313,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,ISIC_0024314,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,ISIC_0024315,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
classes = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
rows = []

for i in range(len(one_hot_labels)):
  label = []
  row = []
  for cls in classes:
    label.append(one_hot_labels.iloc[i][cls])

  row.append(one_hot_labels.iloc[i]['image'])
  row.append(classes[np.argmax(label)])
  rows.append(row)

# print(rows)

In [ ]:
df = pd.DataFrame(rows, columns=['img_id', 'diagnostic'])

In [ ]:
df.head(10)

,img_id,diagnostic
0,ISIC_0024306,NV
1,ISIC_0024307,NV
2,ISIC_0024308,NV
3,ISIC_0024309,NV
4,ISIC_0024310,MEL
5,ISIC_0024311,NV
6,ISIC_0024312,BKL
7,ISIC_0024313,MEL
8,ISIC_0024314,NV
9,ISIC_0024315,MEL


In [ ]:
print(len(df[df['diagnostic'] == 'MEL']))
print(len(df[df['diagnostic'] == 'NV']))
print(len(df[df['diagnostic'] == 'BCC']))
print(len(df[df['diagnostic'] == 'AKIEC']))
print(len(df[df['diagnostic'] == 'BKL']))
print(len(df[df['diagnostic'] == 'DF']))
print(len(df[df['diagnostic'] == 'VASC']))

1113
6705
514
327
1099
115
142


In [ ]:
df.to_csv('ISIC_GroundTruth.csv')

# Data Preprocessing for ISIC 2018 Skin Lesion Classification

**Authors:**
<br>
Akib Mohammed Khan, Alif Asharafee, Fahim Shahriar Khan,
<br>
Islamic University of Techonology.

In [ ]:
import os
import zipfile
import glob
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from shutil import copyfile, move

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16501932769295188467]

###Download Zip

In [ ]:
!gdown --id "1RBzMmkKmz42L1xZpx4_F0Kwn5MN0Rq8c"

Downloading...
From: https://drive.google.com/uc?id=1RBzMmkKmz42L1xZpx4_F0Kwn5MN0Rq8c
To: /content/ISIC2018_Task3_Training_Input.zip
2.77GB [00:36, 76.1MB/s]


In [ ]:
local_zip = '/content/ISIC2018_Task3_Training_Input.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

###Check Directory Length

In [ ]:
print(len(os.listdir('/content/ISIC2018_Task3_Training_Input')))

10017


In [ ]:
txt_files = glob.glob('/content/ISIC2018_Task3_Training_Input/*txt')

for f in txt_files:
  os.remove(f)
print(len(os.listdir('/content/ISIC2018_Task3_Training_Input')))

10015


###Creating File Paths from CSV

In [ ]:
!gdown --id '1qz2SOPbmNlOi0XS8WQb8ZHh_es6c8hvc'

Downloading...
From: https://drive.google.com/uc?id=1qz2SOPbmNlOi0XS8WQb8ZHh_es6c8hvc
To: /content/ISIC_GroundTruth.csv
100% 213k/213k [00:00<00:00, 21.1MB/s]


In [ ]:
files = pd.read_csv('ISIC_GroundTruth.csv')
files.head(5)

,Unnamed: 0,img_id,diagnostic
0,0,ISIC_0024306,NV
1,1,ISIC_0024307,NV
2,2,ISIC_0024308,NV
3,3,ISIC_0024309,NV
4,4,ISIC_0024310,MEL


###File List Generator Function

In [ ]:
def filename_filter(classname):
  return files[files['diagnostic'] == classname]['img_id']

In [ ]:
MEL_filenames = filename_filter('MEL')
NV_filenames = filename_filter('NV')
BCC_filenames = filename_filter('BCC')
AKIEC_filenames = filename_filter('AKIEC')
BKL_filenames = filename_filter('BKL')
DF_filenames = filename_filter('DF')
VASC_filenames = filename_filter('VASC')

###Make Folders to Move Images to their Respective Class Directories

In [ ]:
try:
  os.mkdir("/content/ISIC-full/")
  os.mkdir("/content/ISIC-full/MEL/")
  os.mkdir("/content/ISIC-full/NV/")
  os.mkdir("/content/ISIC-full/BCC/")
  os.mkdir("/content/ISIC-full/AKIEC/")
  os.mkdir("/content/ISIC-full/BKL/")
  os.mkdir("/content/ISIC-full/DF/")
  os.mkdir("/content/ISIC-full/VASC/")
except OSError:
    pass

###Mover Function

In [ ]:
def mover(filenames, dst):
  src = "/content/ISIC2018_Task3_Training_Input"
  count = 0
  for image in filenames:
    image = image + ".jpg"
    move(os.path.join(src, image), os.path.join(dst, image))
    count += 1
  print(f'{count} files moved to {dst}')

In [ ]:
MEL_Destination = "/content/ISIC-full/MEL/"
mover(MEL_filenames, MEL_Destination)

NV_Destination = "/content/ISIC-full/NV/"
mover(NV_filenames, NV_Destination)

BCC_Destination = "/content/ISIC-full/BCC/"
mover(BCC_filenames, BCC_Destination)

AKIEC_Destination = "/content/ISIC-full/AKIEC/"
mover(AKIEC_filenames, AKIEC_Destination)

BKL_Destination = "/content/ISIC-full/BKL/"
mover(BKL_filenames, BKL_Destination)

DF_Destination = "/content/ISIC-full/DF/"
mover(DF_filenames, DF_Destination)

VASC_Destination = "/content/ISIC-full/VASC/"
mover(VASC_filenames, VASC_Destination)

1113 files moved to /content/ISIC-full/MEL/
6705 files moved to /content/ISIC-full/NV/
514 files moved to /content/ISIC-full/BCC/
327 files moved to /content/ISIC-full/AKIEC/
1099 files moved to /content/ISIC-full/BKL/
115 files moved to /content/ISIC-full/DF/
142 files moved to /content/ISIC-full/VASC/


In [ ]:
print(len(os.listdir(MEL_Destination)))
print(len(os.listdir(NV_Destination)))
print(len(os.listdir(BCC_Destination)))
print(len(os.listdir(AKIEC_Destination)))
print(len(os.listdir(BKL_Destination)))
print(len(os.listdir(DF_Destination)))
print(len(os.listdir(VASC_Destination)))

1113
6705
514
327
1099
115
142


###Make Necessary Directories for Train Valid Split

In [ ]:
try:
  os.mkdir("/content/ISIC-full-split/")
  os.mkdir("/content/ISIC-full-split/training/")
  os.mkdir("/content/ISIC-full-split/training/MEL/")
  os.mkdir("/content/ISIC-full-split/training/NV/")
  os.mkdir("/content/ISIC-full-split/training/BCC/")
  os.mkdir("/content/ISIC-full-split/training/AKIEC/")
  os.mkdir("/content/ISIC-full-split/training/BKL/")
  os.mkdir("/content/ISIC-full-split/training/DF/")
  os.mkdir("/content/ISIC-full-split/training/VASC/")
  os.mkdir("/content/ISIC-full-split/validation/")
  os.mkdir("/content/ISIC-full-split/validation/MEL/")
  os.mkdir("/content/ISIC-full-split/validation/NV/")
  os.mkdir("/content/ISIC-full-split/validation/BCC/")
  os.mkdir("/content/ISIC-full-split/validation/AKIEC/")
  os.mkdir("/content/ISIC-full-split/validation/BKL/")
  os.mkdir("/content/ISIC-full-split/validation/DF/")
  os.mkdir("/content/ISIC-full-split/validation/VASC/")
except OSError:
    pass

###Validation Split

In [ ]:
def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
  source_files = os.listdir(SOURCE)
  randomized_source_files = random.sample(source_files, len(source_files))

  train_split = randomized_source_files[ :int(SPLIT_SIZE * len(source_files))]
  valid_split = randomized_source_files[int(SPLIT_SIZE * len(source_files)): ]

  for file in train_split:
      if os.path.getsize(os.path.join(SOURCE, file)) != 0:
          move(os.path.join(SOURCE, file), os.path.join(TRAINING, file))

  for file in valid_split:
      if os.path.getsize(os.path.join(SOURCE, file)) != 0:
          move(os.path.join(SOURCE, file), os.path.join(VALIDATION, file))

MEL_Dir = "/content/ISIC-full/MEL/"
MEL_Training = "/content/ISIC-full-split/training/MEL/"
MEL_Testing = "/content/ISIC-full-split/validation/MEL/"

NV_Dir = "/content/ISIC-full/NV/"
NV_Training = "/content/ISIC-full-split/training/NV/"
NV_Testing = "/content/ISIC-full-split/validation/NV/"

BCC_Dir = "/content/ISIC-full/BCC/"
BCC_Training = "/content/ISIC-full-split/training/BCC/"
BCC_Testing = "/content/ISIC-full-split/validation/BCC/"

AKIEC_Dir = "/content/ISIC-full/AKIEC/"
AKIEC_Training = "/content/ISIC-full-split/training/AKIEC/"
AKIEC_Testing = "/content/ISIC-full-split/validation/AKIEC/"

BKL_Dir = "/content/ISIC-full/BKL/"
BKL_Training = "/content/ISIC-full-split/training/BKL/"
BKL_Testing = "/content/ISIC-full-split/validation/BKL/"

DF_Dir = "/content/ISIC-full/DF/"
DF_Training = "/content/ISIC-full-split/training/DF/"
DF_Testing = "/content/ISIC-full-split/validation/DF/"

VASC_Dir = "/content/ISIC-full/VASC/"
VASC_Training = "/content/ISIC-full-split/training/VASC/"
VASC_Testing = "/content/ISIC-full-split/validation/VASC/"

split_size = .80

split_data(MEL_Dir, MEL_Training, MEL_Testing, split_size)
split_data(NV_Dir, NV_Training, NV_Testing, split_size)
split_data(BCC_Dir, BCC_Training, BCC_Testing, split_size)
split_data(AKIEC_Dir, AKIEC_Training, AKIEC_Testing, split_size)
split_data(BKL_Dir, BKL_Training, BKL_Testing, split_size)
split_data(DF_Dir, DF_Training, DF_Testing, split_size)
split_data(VASC_Dir, VASC_Training, VASC_Testing, split_size)

In [ ]:
print('MEL -> Training:', len(os.listdir(MEL_Training)), 'Testing:', len(os.listdir(MEL_Testing)))
print('NV -> Training:', len(os.listdir(NV_Training)), 'Testing:', len(os.listdir(NV_Testing)))
print('BCC -> Training:', len(os.listdir(BCC_Training)), 'Testing:', len(os.listdir(BCC_Testing)))
print('AKIEC -> Training:', len(os.listdir(AKIEC_Training)), 'Testing:', len(os.listdir(AKIEC_Testing)))
print('BKL -> Training:', len(os.listdir(BKL_Training)), 'Testing:', len(os.listdir(BKL_Testing)))
print('DF -> Training:', len(os.listdir(DF_Training)), 'Testing:', len(os.listdir(DF_Testing)))
print('VASC -> Training:', len(os.listdir(VASC_Training)), 'Testing:', len(os.listdir(VASC_Testing)))

MEL -> Training: 890 Testing: 223
NV -> Training: 5364 Testing: 1341
BCC -> Training: 411 Testing: 103
AKIEC -> Training: 261 Testing: 66
BKL -> Training: 879 Testing: 220
DF -> Training: 92 Testing: 23
VASC -> Training: 113 Testing: 29


In [ ]:
!zip -r ISIC-split.zip ISIC-full-split

Streaming output truncated to the last 5000 lines.
  adding: ISIC-full-split/training/NV/ISIC_0032950.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0027459.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0030820.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0025496.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0028090.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0026669.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0029379.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0026087.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0034257.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0024543.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0030920.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0028538.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0027617.jpg (deflated 0%)
  adding: ISIC-full-split/training/NV/ISIC_0028949.j

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ISIC-split.zip '/content/drive/MyDrive/Thesis Skin Lesion/ISIC2018/'